### Установки

In [1]:
!pip install pymorphy2
%pip install spacy
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 57.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=04f989692737630fbca0147fe68194a852d886ba66b662ccb623182e71c8b08f
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-10 16:06:38.335170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

### Импорт

In [2]:
from pymorphy2 import MorphAnalyzer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from random import randint
from pprint import pprint

import pandas as pd
import numpy as np
import pickle
import nltk
import json
import gc
import re



*   распаковка json
*   знакомство с данными
*   разбиение текстов на  предложения
*   лемматизация и очищение от стоп-слов



### распаковка json, приведение к удобному виду для pandas

In [4]:
with open('/content/train.json', 'r', encoding='utf-8') as f:
  json_data = json.load(f)
len(json_data)

FileNotFoundError: ignored

In [ ]:
json_data[0]['extracted_part']

In [ ]:
json_data[0]['text'][1279:1343]


In [ ]:
lenght = len(json_data)
for i in range(lenght):
  target = json_data[i]['extracted_part']['text'][0]
  if target:
    json_data[i]['target'] = target

  else: 
    json_data[i]['target'] = np.nan

  del json_data[i]['extracted_part']
  
json_data[0]

In [ ]:
with open('train_unpacked.json', 'w') as f:
  json.dump(json_data, f)

In [ ]:
del json_data
gc.collect()

### откроем в pandas посмотрим на данные

In [51]:
df = pd.read_json('/content/train_unpacked.json', encoding='utf-8')
df.sample(3)

,id,text,label,target
164,621939083,13 ПРОЕКТ КОНТРАКТ №____ на поставку товаров г...,обеспечение исполнения контракта,Размер обеспечения исполнения Контракта состав...
217,56810761,32 УТВЕРЖДАЮ Начальник департамента городского...,обеспечение гарантийных обязательств,Размер обеспечения гарантийных обязательств по...
183,981282799,Извещение о проведении запроса котировок в эле...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 24578....


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1799 non-null   int64 
 1   text    1799 non-null   object
 2   label   1799 non-null   object
 3   target  1492 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


Посмотрим поближе на сами тексты

In [53]:
df['label'].value_counts()

обеспечение исполнения контракта        988
обеспечение гарантийных обязательств    811
Name: label, dtype: int64

In [54]:
string = randint(0, 1799)
df.loc[string, 'text'], df.loc[string, 'label'], df.loc[string, 'target'] 

('Утверждена: Начальник МКУ «ХЭГ» ______________ Шигорин С. В наименование и описание объекта закупки, условия контракта (РАЗДЕЛ 2 ИЗВЕЩЕНИЯ ОБ ОСУЩЕСТВЛЕНИИ ЗАКУПКИ) 1 Информация о заказчике Бюджетополучатель: Муниципальное казенное учреждение «Хозяйственно-эксплуатационная ыть только субъекты малого предпринимательства, социально ориентированные некоммерческие организации. 14 Требование, установленное в соответствии с частью 5 статьи 30 Федерального закона от 05.04.2013 №44-ФЗ, с указанием в соответствии с частью 6 статьи 30 Федерального закона от 05.04.2013 № 44-ФЗ объема привлечения к исполнению контрактов субподрядчиков, соисполнителей из числа субъектов малого предпринимательства, социально ориентированных некоммерческих организаций Не установлено 15 Условия, запреты и ограничения допуска товаров, происходящих из иностранного государства или группы иностранных государств (в соответствии со ст. 14 Федерального закона от 05.04.2013 № 44-ФЗ) Не установлено 16 Установить обеспечение 

В текстах не везде расставлены знаки препинания, поэтому мы не можем чётко разделить предложения и можем ориентироваться только на регистр

### разбиение текстов 

In [55]:
df.sample(4)

,id,text,label,target
1166,456169402,8 Приложение №4 к извещению об электронном аук...,обеспечение гарантийных обязательств,Обеспечение гарантийных обязательств установле...
1154,711395010,УТВЕРЖДАЮ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ХАНТЫ-МАНСИЙСКО...,обеспечение гарантийных обязательств,Размер обеспечения гарантийных обязательств 13...
638,201962501,Утверждаю: Заведующий структурным подразделени...,обеспечение исполнения контракта,Размер обеспечения исполнения договора 5% от н...
1360,345442312,11 21 УТВЕРЖДАЮ Директор СПб ГБУ СОН «КЦСОН Фр...,обеспечение гарантийных обязательств,Размер обеспечения гарантийных обязательств по...


Сложность составляют слова, которые в юридических документах пишутся с заглавной буквы: Заказчик, Договор, Сторон и т.д.

Попробуем написать функцию для разделения текстов на предложения

In [56]:
def lower_the_unnessesary_words(text):
  # сделаем слова, которые пишутся с заглавной буквы и не стоят в начале предложения прописными
  capital_words_exceptions = [
      'Заказчик', 'Получател', 'Договор', 'Поставщик', 
      'Федер', 'Закон', 'Росси', 'Контракт', 'Гражданск', 
      'Министерств', 'Директор', 'Сторон', 'Оборудован', 
      'Акт', 'МВД', 'НМЦК'
  ]
  splitted_text = text.split(' ')
  for i in range(len(splitted_text)):
    for exception in capital_words_exceptions:  
      if type(splitted_text[i]) is str and splitted_text[i].startswith(exception):
        splitted_text[i] = splitted_text[i].lower()

  return ' '.join(splitted_text)

In [57]:
def get_back_lowered_words(text):
  capital_words_exceptions = [
      'заказчик', 'получател', 'договор', 'поставщик', 
      'федер', 'закон', 'росси', 'гражданск', 
      'министерств', 'директор', 'сторон', 'оборудован', 
      'акт'
  ]
  all_capital_words = ['мвд', 'нмцк']

  splitted_text = text.split()
  for i in range(len(splitted_text)):
    for exception in capital_words_exceptions:  
      if type(splitted_text[i]) is str and splitted_text[i].startswith(exception):
        splitted_text[i] = splitted_text[i].capitalize()

      if splitted_text[i] in all_capital_words:
        splitted_text[i] = splitted_text[i].upper()
  return ' '.join(splitted_text)

In [58]:
def text_separation_by_sentences(text, min_sentence_lenght=20):
  # сначала разделим текст по заглавным и уберём неинформативные предложения
  sentences = []
  text = text.replace('\n', ' ')
  text = lower_the_unnessesary_words(text)
  raw_sentences = re.split(r'\s+(?=[А-Я])', text)
  for i in range(len(raw_sentences)):
    raw_sentences[i] = get_back_lowered_words(raw_sentences[i])

  for i in raw_sentences:
    if len(i) > min_sentence_lenght:
      sentences.append(i)

  return sentences

Добавим к нашему разбиению на предложения фрагменты, разделения запятыми и тире.

In [59]:
def add_comma_separated_fragments(text:list):
  result = []
  for sentence in text:
    result.extend(re.split(',|-|_', sentence.strip()))
  for i in range(len(result)):
    result[i] = result[i].strip()
  text.extend(result)
  return text

Наш разделитель не будет идеально делить предложения, т.к. могут быть предложения, у которых в начале стоит одно из слов-исключений, либо когда мы не внесли слово в список слов-исключений, но оно написано с большой буквы 

In [60]:
for i in range(len(df)):
  sentences = text_separation_by_sentences(df.loc[i, 'text'])
  df.at[i, 'text'] = sentences

In [61]:
df.to_json('./train_with_splitted_texts.json')

также spacy для этой задачи работает довольно медленно, т.к. выполняет не только операцию разбиения на предложения, а также использует 

In [62]:
for i in range(len(df)):
  sentences = add_comma_separated_fragments(df.loc[i, 'text'])
  df.at[i, 'text'] = sentences


In [63]:
df.to_json('./train_with_splitted_texts_extended.json')

In [64]:
string = randint(0, 1799)
df.loc[string, 'text'], df.loc[string, 'label'], df.loc[string, 'target'] 

(['Утверждено на заседании',
  'Постоянно действующей закупочной',
  'Протокол № 61 от 08.09.2022г. ризнанным соответствующим требованиям извещения о закупке) относительно',
  'НМЦ, указанной Заказчиком в извещении о проведении запроса цен, на 25 и более процентов.',
  'При расчете процента снижения для установления факта аномально низкой цены (демпинга) округление производится до целого числа по математическим правилам с учетом двух знаков после запятой. 9.2.',
  'В случае если при проведении закупки победителем (либо единственным участником, признанным соответствующим требованиям извещения о закупке) признается участник, предложивший аномально низкое ценовое предложение, таким участником предоставляется обеспечение исполнения Договора в следующем размере: №',
  'Матрица Договорных условий',
  'Размер обеспечения исполнения Договора в случае подачи участником закупки аномально низкого ценового предложения',
  'Требование по обеспечению исполнения Договора',
  'Обеспечение на возврат а

Алгоритм справляется с разбиением предложений достаточно хорошо, качество разбиения можно повысить, добавляя и убирая слова-исключения. 

Ещё одна проблема заключается в том, что в таргете у нас не всегда стоят законченные предложения, что может сказаться на accuracy. Например:


*   Участник закупки, с которым заключается договор, обязан представить обеспечение исполнения договора в размере 5% от цены

Полное предложение: "*Участник закупки, с которым заключается договор, обязан представить обеспечение исполнения договора в размере 5% от цены, по которой в соответствии с законом заключается контракт  17 1  заказчик вправе установить требование обеспечения гарантийных обязательств в случае установления требований к таким обязательствам в соответствии с частью 4 статьи 33 настоящего федерального закона*"

Но и есть и такие конктрукции в таргете:


*   Обеспечение гарантийных обязательств представляется в размере 127 598,00 руб. , что составляет 2 % от начальной (максимальной) цены Договора, предусмотренной документацией о закупке.
*  Размер обеспечения исполнения контракта, гарантийных обязательств 5% от НМЦК в сумме – 683 (шестьсот восемьдесят три) рубля 33 копейки.




### Очистка текста и лемматизация

In [ ]:
df = pd.read_json('/content/train_with_splitted_texts.json', encoding='utf-8')
df.head()

Функция для очистки лишних знаков препинания, чисел, а также букв английского алфавита

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatization(text):
  pattern = "[A-Za-z0-9!#$%&'()*+№,./:;<=>?@[\]^_`{|}~—\"\-]+"
  text = re.sub(pattern, ' ', text)
  tokens = []
  for token in text.split():
      if token and token not in stopwords_ru:
          token = token.strip()
          token = morph.normal_forms(token)[0]
          tokens.append(token)
  return tokens

In [ ]:
sentence = df.loc[4, 'text'][0]
sentence

In [ ]:
lemmatization(sentence)

In [ ]:
%%time
for i in range(len(df)):
  lemmatized_text = []

  for k in range(len(df.loc[i, 'text'])):
    sentence = df.loc[i, 'text'][k]
    sentence =  lemmatization(sentence)
    lemmatized_text.append(sentence)
  df.at[i, 'text'] = lemmatized_text
   

In [ ]:
df.to_json('train_lemmatized.json')

### Сохранение доп. датасетов для классификации.

In [ ]:
contract_enforcement = df[df['label'] == 'обеспечение исполнения контракта']['target'].values
garantee_enforcement = df[df['label'] == 'обеспечение гарантийных обязательств']['target'].values

In [69]:
nonetype_sentences = df[df['target'].isna()]['text'].values

также сохраним таргеты двух классов для модели classy classification

In [ ]:
with open('contract_enforcement.pkl', 'wb') as fp:
  pickle.dump(contract_enforcement, fp)

In [ ]:
with open('garantee_enforcement.pkl', 'wb') as fp:
  pickle.dump(garantee_enforcement, fp)

In [71]:
with open('nonetype.pkl', 'wb') as fp:
  pickle.dump(nonetype_sentences, fp)

### улучшение разбиения текста на предложения 

Цель - добиться, чтобы в каждом разбиении присутствовал фрагмент соответствующий таргету. Это поможет сильно улучшить accuracy

In [108]:
df = pd.read_json('/content/train_unpacked.json', encoding='utf-8')

In [109]:
random_string = randint(0, 1799)
df.loc[random_string, :].values

array([323658138,
       '1 Приложение № 4 к извещению о проведении открытого конкурса в электронной форме ПРОЕКТ ГОСУДАРСТВЕННОГО КОНТРАКТА № _____ на выполнение работ для государственных нужд Астраханской области г. Астрахань _________________ г. 1 Приложение № 4 к извещению о проведении открытого конкурса в электронной форме ПРОЕКТ ГОСУДАРСТВЕННОГО КОНТРАКТА № _____ на выполнение работ для государственных нужд Астраханской области г. Астрахань _________________ г. Государственное казенное учреждение Астраханской области «Управление автомобильными дорогами общего пользования «Астраханьавтодор», именуемое в дальнейшем «Заказчик», в лице _____________________________, действующего (-ей) на основании Устава, с одной стороны, заключило от имени Астраханской области в целях обеспечения государственных нужд Астраханской области с _________________________________________________________________________________, (полное наименование «Исполнителя») именуемым в дальнейшем «Исполнитель», в лиц

Зададим несколько правил для разбиения:

*   Если в предложении на конце стоит знак, его нужно сохранить

*   Предложение не должно оканчиваться на цифры с точками (3.4.)

*   Если встречаются слова с большой буквы в предложении - они  должны остаться с большой буквы

*   Если фрагмент оканчивается на , или - тогда в конце знак не ставится


*   Чтобы отследить фрагменты, которые не разделены знаками препинания, или заглавными буквами, можно составить список слов, на которых скорее всего оканчивается искомое предложение



Таким образом, придётся расширить исходные тексты и хранить ещё и фрагменты разделений по знакам , и -

In [110]:
def is_in_stop_list(word):
  stop_list = ['заказчик', 'получател', 'договор', 'поставщик', 
      'федер', 'закон', 'росси', 'гражданск', 
      'министерств', 'директор', 'сторон', 'оборудован', 
      'акт', 'мвд', 'нмцк', 'удмур', 'глав', 'москв', 'контракт', 
      'исполнител', 'ндс', 'правительств', 'рф', 'документ', 
      'продукци', 'проект']

  for exception in stop_list:
    word = word.lower()
    if word.startswith(exception):
      return True
  return False

In [111]:
def is_common_target_end(sentence):
  # в стоп листе сохраним падежи, чтобы не генерить много фрагментов
  stop_list = [
      'контракта', 'нцмд', 'копейки'
  ]

  index_list = []

  word_list = sentence.split()
  for word in word_list:
    if word.lower() in stop_list:
      index_list.append(sentence.index(word) + len(word))


  return index_list

In [112]:
def comma_separate(list_of_sentences, min_sentence_lenght=15):
  result_fragments_list = []

  def separate(sentence):
    list_of_commas_index = []
    list_of_comma_separated_fragments = []
    for i in range(len(sentence)):
      try:
        if sentence[i] == ',' or sentence[i] == '-':
          if not sentence[i-1].isnumeric() and not sentence[i-1].isupper() and not sentence[i+1].isnumeric() and not sentence[i+1].isupper():
            list_of_commas_index.append(i)
      except:
        pass

    list_of_commas_index.extend(is_common_target_end(sentence))
    start = 0
    for end in list_of_commas_index:
      fragment = sentence[start:end]
      list_of_comma_separated_fragments.append(fragment)
    
    return list(set(list_of_comma_separated_fragments))

  
  for sentence in list_of_sentences:
    if len(sentence) > min_sentence_lenght:
      result_fragments_list.extend(separate(sentence))

  return result_fragments_list

In [113]:
def check_the_last_word(list_of_words):
  try: 
    if int(list_of_words[-1]) < 100:
      return list_of_words[:-1]
  except:
    if '<' in list_of_words[-1] and '>' in list_of_words[-1]:
      return list_of_words[:-1]
  return list_of_words

In [114]:
def splitter(text, min_sentence_lenght=6):
  list_of_sentences = []
  list_of_words = text.split()
  sentence = []
  previous_word = ''
  for word in list_of_words:
    try:
      if not len(word) == 1:
        # критерии начала следующего предложения
        if word[-1] == '.' and word[-2].isnumeric():
          sentence = check_the_last_word(sentence)
          list_of_sentences.append(' '.join(sentence))
          sentence = []
        if word[0].isupper() and not word.isupper():
          if not is_in_stop_list(word) and sentence:
            sentence = check_the_last_word(sentence)
            list_of_sentences.append(' '.join(sentence))
            sentence = []
        if is_in_stop_list(word) and previous_word[-1] == '.':
          sentence = check_the_last_word(sentence)
          list_of_sentences.append(' '.join(sentence))
          sentence = []

      else:
        if word.isupper():
          sentence = check_the_last_word(sentence)
          list_of_sentences.append(' '.join(sentence))
          sentence = []

    except:
      pass
    sentence.append(word)

    if word is list_of_words[-1]:
      sentence = check_the_last_word(sentence)
      list_of_sentences.append(' '.join(sentence))
    previous_word = word

  # добавим фрагменты к списку предложений если extended=True
  # if extended:
  list_of_sentences.extend(comma_separate(list_of_sentences))

  for sentence in list_of_sentences:
    if len(sentence) < min_sentence_lenght or not sentence:
      del list_of_sentences[list_of_sentences.index(sentence)]
  return list_of_sentences
      

In [115]:
random_string = randint(0, 1799)
sample_text = df.loc[random_string, 'text']

In [116]:
splitted = splitter(sample_text)
for i, value in enumerate(splitted):
  if value:
    print(f'{i+1}) {value}')

print(df.loc[random_string, 'target'], df.loc[random_string, 'target'] in splitted)

1) Извещение о проведении электронного аукциона для закупки №0809500000322002133
2) Общая информация
3) Номер извещения 0809500000322002133
4) Наименование объекта закупки
5) Приобретение жилого помещения (Квартиры) для обеспечения детей-сирот, детей, оставшихся без попечения родителей, лиц из ение - НБ
6) Республика
7) Мордовия
8) Банка России//УФК по
9) Республике
10) Мордовия г.
11) Саранск "Номер корреспондентского счета" 40102810345370000076
12) Требования к гарантии качества товара, работы, услуги
13) Требуется гарантия качества товара, работы, услуги
14) Информация о требованиях к гарантийному обслуживанию товара
15) Не установлено
16) Требования к гарантии производителя товара
17) Не установлено
18) Срок, на который предоставляется гарантия
19) Гарантийный срок на жилое помещение, за исключением элементов чистовой отделки, технологического и инженерного оборудования, входящего в состав жилого помещения, составляет не менее трех лет.
20) Гарантийный срок на технологическое и инж

Помотрим, как хорошо мы разделили предложения: для этого напишем функцию, которая будет считать процент вхождения таргета в разбиение. 

In [117]:
def division_check(df:pd.DataFrame):
  counter = 0
  try:
    df.drop(columns=['not_found'], inplace=True)
  except:
    pass

  for i in range(len(df)):
    try:
      if df.loc[i, 'target'] in splitter(df.loc[i, 'text']):
        counter += 1
      else:
        df.loc[i, 'not_found'] = 0
    except IndexError:
      print(df.loc[i, 'text'])
      print(i)
  return counter / 1492, df[df['not_found']==0].reset_index(drop=True)  # 1492 - ненулевые значения в таргете

In [118]:
def split_df_texts_by_sentences(data_frame:pd.DataFrame):
  for i in range(len(data_frame)-1):
    splitted = splitter(data_frame.loc[i, 'text'])
    data_frame.at[i, 'text'] = splitted
  return data_frame


In [119]:
precision, not_found = division_check(df)

In [120]:
precision

0.631367292225201

In [121]:
df_splitted = split_df_texts_by_sentences(df)

In [122]:
df_splitted.to_json('train_with_splitted_texts_extended.json')

In [123]:
not_found.head()

,id,text,label,target,not_found
0,462513308,Приложение № 3 ГОСУДАРСТВЕННЫЙ КОНТРАКТ (ПРОЕК...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта устана...,0.0
1,988172508,Идентификационный код закупки: 222637800390863...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0
2,359099757,Идентификационный код закупки: 223631102474263...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0
3,967226407,Идентификационный код закупки: 222633500240263...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0
4,19349887,Идентификационный код закупки: 222634000559063...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0


In [124]:
random_str = randint(0, len(not_found)-1)
not_found_split = splitter(not_found.loc[random_str, 'text'])
for i in not_found_split:
  print(i)
print('target:')
not_found.loc[random_str, 'target'] 

УТВЕРЖДАЮ
Генеральный директор АО «АТА» ________ А.В.
Балебанов «___» ________ 2022 года ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ
Открытый одноэтапный конкурс в электронной форме без предварительного квалификационного отбора на поставку автомобильного топлива по пластиковым картам для АО «АТА» г.
Москва том 1 «общая и коммерческая части» СОГЛАСОВАНО:
Начальник отдела закупок _________________ С.Г.
Гаврилов 2021
Нагатинская улица, д.4а, Москва, 115230
Тел.: (495) 645-82-77, (495) 645-82-66
Факс: (499) 611-30-20,
E-mail: info@oaoata.ru ОГРН 1057749068002, ИНН/КПП 7724560930/772401001 СОДЕРЖАНИЕ ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ 1 СОДЕРЖАНИЕ
1. ИЗВЕЩЕНИЕ
О ПРОВЕДЕНИИ ЗАКУПКИ 3 ЧАСТЬ 1
2. ТРЕБОВАНИЯ.
ДОКУМЕНТЫ. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ
В ЗАКУПКЕ.
2.1. ТРЕБОВАНИЯ.
ДОКУМЕНТЫ, ПОДТВЕРЖДАЮЩИЕ СООТВЕТСТВИЕ УСТАНОВЛЕННЫМ ТРЕБОВАНИЯМ.
2.1.1.
Требования к участникам закупки
2.1.2.
Требования к продукции
2.1.3.
Требования к банкам-гарантам, предоставляющим обеспечение заявки
1.1. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ
В ЗА ЧАСТЬ 3 Проект 

создадим файл utils.py, куда запишем фукнции сплита и подсчёта accuracy. далее будем пробовать различные модели